# Metabolic network analysis using *volestipy*

Aim of this notebook is to present the complete pipeline for random sampling in a metabolic network using the ```volesitpy``` library. As input, we will use a BIGG model of human erythrocyte metabolism ([iAB-RBC-283](http://bigg.ucsd.edu/models/iAB_RBC_283)) in its ```.json``` format. 

This model consists of 342 metabolites and 469 reactions. 

Other formats, such as ```.mat``` are supported by ```volestipy``` but this is not in the scope of this tutorial. 

**Attention!**
This tutorial assumes that you have already compiled the ```volestipy``` library, following the steps described [here](https://github.com/hariszaf/volume_approximation_bio/tree/develop/volestipy).

## Dependencies

With respect to this ```jupyter notebook```.
First you need to create a **conda environment** by making use of at least Python 3.6. 
Then open the notebook using the ```jupyter notebook``` command after entering the conda evironment you built. 

For example, considering that the base environment of ```conda``` includes Python 3.6:

```conda activate```

```jupyter notebook```

To use Jupyter with a certain conda environment we also need to run the followings:

````conda install ipykernel
ipython kernel install --name MY_CONDA_ENV --user
````

Before showing how you can exploit the *volestipy* software, we first need to get all the relative dependencies. 

This demo uses [Anaconda](https://www.anaconda.com/products/individual) which you can download following [these](https://www.digitalocean.com/community/tutorials/how-to-install-anaconda-on-ubuntu-18-04-quickstart) instructions.

Furtheremore, special, powerful mathematical optimization solvers like [Gurobi](https://www.gurobi.com/) are also used. You can get Gurobi following the steps described [here](https://support.gurobi.com/hc/en-us/articles/360044290292-Installing-Gurobi-for-Python). Keep in mind that you will need a Gurobi license. To do this, you need to create a Gurobi user account and then follow the instructions for a license you will find there.

The main libraries you need to run this pipeline are the following:

* ```numpy```
* ```gurobipy``` and of course
* ```volestipy```

You will also need a library for plotting, like ```matplotib``` but this is up to you!

To get any libraries that need to run commands as ```sudo``` you need to make a file including **only** your password and replace ```/home/haris/Desktop/running/metabolic_network_pipeline_volestipy/my_project_virtual_env/error.txt``` with the corresponding path. 

In order to make possible to install what is needed in this conda evironment we use the ```getpass``` library that prompts the user for a password without echoing.

In [ ]:
import sys
import getpass 
import os

So, with respect to the ```numpy``` library, let us first get it in our conda environment. 
This is going to take a while. 

In [ ]:
!conda install --yes --prefix {sys.prefix} numpy

Now we can import it. 

In [ ]:
import numpy as np

Assuming you have already ```gurobipy``` on your computer, you may just add its path on your conda environment with the following command.

In [ ]:
sys.path.append('/usr/local/lib/python3.6/dist-packages/gurobipy/')

Otherwise, you can install ```gurobipy``` on the conda enviroment directly. 

In [ ]:
# Get GUROBI through anaconda - You can find more about installing Gurobi here: 
# https://support.gurobi.com/hc/en-us/articles/360044290292-Installing-Gurobi-for-Python
!conda install -y -c gurobi gurobi
print("*** The Gurobi solver library has now been installed *** \n\n")

After executing one of the above ways to get `gurobipy`, import this library and check if everything is working fine. 

In [ ]:
import gurobipy as gp
#from gurobipy import GRB

# This is just a test that the Gurobi solver is well installed

# Create a new model
m = gp.Model("mip1")

print("\n*** Gurobi test has been completed successfully. ***\n")

Likewise, for the `scipy` library. Install it through conda on this environment.

In [ ]:
!conda install --yes --prefix {sys.prefix} scipy

We do not need to import `scipy` here, however the `volestipy` does that. 

Finally, we have to compile and then import the ```volestipy``` library. To this end, we first need to get the `cython` library.

In [ ]:
pip install Cython

And then run the following command as it was from our terminal. It is the `!` that allows for the latter.

In [ ]:
!LDFLAGS="-L/usr/lib/lp_solve/" python3 setup.py install --user

In [ ]:
from volestipy import *

Now we are ready to read our network BIGG file and run the necessary steps to get our random points from its correspoding polytope.

## Read your network file

We have downloaded our BIGG model in its ```.json``` format. We keep this file as a variable. 

In [ ]:
input_file = '/home/haris/bigg_files/iAB_RBC_283.json'

And we run the ```volestipy``` function to read it.

In [ ]:
human_erythrocyte_met_net = read_json_file(input_file)

You may see the whole BIGG file, simply by printing the ```human_erythrocyte_met_net``` variable. 

In [ ]:
human_erythrocyte_met_net

As the ```human_erythrocyte_met_net``` variable is a tuple, we can get its different features in a straight forward way:

In [ ]:
A = human_erythrocyte_met_net[0]
b = human_erythrocyte_met_net[1]
Aeq = human_erythrocyte_met_net[2]
beq = human_erythrocyte_met_net[3]

In the ```Aeq``` variable is the stoichiometric matrix of the metabolic network under study, while in the ```beq``` variable we have its steady state. The variables ```A``` and ```b``` allow us to represent the upper and the lower bounds of the reactions.

Let's have a look on these variables. ```Aeq``` our stoichiometric matrix has dimensions:

In [ ]:
Aeq.shape

meaning we have 342 metabolites that take part in 469 reactions. So, our network is built from these 95 reactions and their corresponding fluxes.

And let us also keep the reactions of the network in a variable. 

In [ ]:
reactions = human_erythrocyte_met_net[5]

Let us assume that we are interested in one of the reactions that we saw before in the glycosylis path; the one that uses Hexokinase(HEX1) and is responsible for the following reaction:
atp_c + glc__D_c ⇌ adp_c + g6p_c + h_c

We first need to find its index on the reactions list.

In [ ]:
reactions.index('HEX1')

And then keep it as a variable.

In [ ]:
hexokinase = reactions[218]

Now we can proceed in the necessary pre-processing steps for getting the polytope that derives from this metabolic network. 

## Preprocess

In terms of making things faster, we run a pre process step in order to run our pipeline in a more efficient, from a computational point of view, way.

In [ ]:
proc = pre_process(A, b, Aeq, beq)

From this, we get the processed A, b, Aeq and beq and we keep them in distinct variables correspodingly.

In [ ]:
A_proc = proc[0]
b_proc = proc[1]
Aeq_proc = proc[2]
beq_proc = proc[3]

In case you the ```pre_process``` step had already run, you may load its output using the following commands:

In [ ]:
A_proc = np.load('/home/haris/Documents/GitHub/volesti_fork/volestipy/tests/A_preprocessed.npy')
b_proc = np.load('/home/haris/Documents/GitHub/volesti_fork/volestipy/tests/b_preprocessed.npy')
Aeq_proc = np.load('/home/haris/Documents/GitHub/volesti_fork/volestipy/tests/Aeq_preprocessed.npy')
beq_proc = np.load('/home/haris/Documents/GitHub/volesti_fork/volestipy/tests/beq_preprocessed.npy')

In [ ]:
Aeq_proc.shape

## Full dimensional (not always required step)

Now we are able to use the pre processed polytope to get the full dimensional polytope that derives from our initial one. To this end we first build an object for the ```low_dim_HPolytope``` class for the pre-processed polytope. 

In [ ]:
low_hp = low_dim_HPolytope(A_proc, b_proc, Aeq_proc, beq_proc)

And then we run the ```full_dimensiolal_polytope``` function of ```volestipy``` to get it. 

In [ ]:
get_fd_hp = low_hp.full_dimensiolal_polytope()

A_fd = get_fd_hp[0].A
b_fd = get_fd_hp[0].b
N = get_fd_hp[1]
N_shift = get_fd_hp[2]

Once we have the full dimensional polytope, we are able to get the max ball of that.

In [ ]:
max_ball_center_point, max_ball_radius = get_max_ball(A_fd, b_fd)

Now we can use this max ball for rounding the full dimensional polytope. 

## Rounding

Using the A and b of the full dimensional polytope, we build a new HPolytope object.

In [ ]:
hp = HPolytope(A_fd, b_fd)

And we use the max ball to round it.

In [ ]:
rounding_output_max_ellipsoid = hp.rounding_svd()

We can now get the features or the rounded polytope.

In [ ]:
rounded_A = rounding_output_max_ellipsoid[0]
rounded_b = rounding_output_max_ellipsoid[1]
rounded_T = rounding_output_max_ellipsoid[2]
rounded_shift = rounding_output_max_ellipsoid[3]

And use the to get the max ball of this full dimensional, rounded polytope.

In [ ]:
rounded_center_point, rounded_radius = get_max_ball(rounded_A, rounded_b)

Let's have a look at that:

In [ ]:
rounded_center_point

In [ ]:
rounded_radius

## Sampling

Finally, we build the full dimenionsal rounded polytope. 

In [ ]:
rounded_polytope = HPolytope(rounded_A, rounded_b)

Keep the dimension of the rounded polytope

In [ ]:
d = rounded_polytope.dimensions
d

And calculate the value of the L parameter for the sampling function according the following formula:

In [ ]:
L_value = 4 * d * rounded_radius

And using the latter max ball, we sample on it

In [ ]:
samples = rounded_polytope.generate_samples(walk_len = 5, 
                                            number_of_points = 10000, 
                                            number_of_points_to_burn = 50, 
                                            radius = rounded_radius, 
                                            inner_point = rounded_center_point,
                                            L = L_value)

And these are the points sampled: 

In [ ]:
samples

However, these samples "live" in the world of the rounded, full dimensional polytope. Thus, we need to map them back in our initial polyopte. To do that we run one last ```volestipy``` function, using the features returned from the ```rounding``` and the ```full_dimensional``` functions. 

In [ ]:
mapped_samples = map_samples_on_initial_polytope(samples, rounded_T, rounded_shift, N, N_shift)

Let's have a look on our final points and their dimension. 

In [ ]:
mapped_samples.shape

## Analysis, plots etc.

We will use the ```matplotlib``` library to plot our random samples. First, we download it to our environment.

In [ ]:
pip install matplotlib

Now we can import it.

In [ ]:
import matplotlib.pyplot as plt

We mentioned that we are insterested in the HEX1 reaction that we have already kept as a variable and that we saw that its index is 218 on the reactions list.

We keep all the fluxes' values from the points sampled for this reactions in a variable. And we print its shape to be sure that we have got this right.

In [ ]:
fluxes = mapped_samples[218,:]

Now we can get the minimum and the maximum value of ETOHt2r flux.

In [ ]:
maxElement = np.amax(fluxes)
minElement = np.amin(fluxes)
maxElement, minElement

And now we may plot our samples! 

In [ ]:
x = np.array([x for x in range(samples.shape[0])])
plt.plot(x, fluxes)
plt.show()

And also as an histogram!

In [ ]:
plt.hist(fluxes, bins='auto', histtype='bar', rwidth=0.7)
plt.show

## Extras

In case you need to work with ```.mat``` files you need to implement some further steps that are not in the interest of this tutorial. 

To read a ```.mat``` file you need some extra Python libraries, as it is quite a challenging task. If you are more interested in that, you can read this article [here](https://scipy-cookbook.readthedocs.io/items/Reading_mat_files.html).

In [ ]:
## Get the h5py library in case you are working with .mat files of 7.3 release of Matlab and after 
#!sudo -H -S pip install h5py < /home/haris/Desktop/running/metabolic_network_pipeline_volestipy/\
#my_project_virtual_env/error.txt
#print("*** The h5py library has now been installed *** \n\n")

In [ ]:
## Get the tables library to read .mat files
#!sudo -H -S pip3 install tables < /home/haris/Desktop/running/metabolic_network_pipeline_volestipy\
#/my_project_virtual_env/error.txt
#print("*** The tables library has now been installed *** \n\n")

In [ ]:
## Matlab up to 7.1 = mat files created with Matlab up to version 7.1 can be read using the mio module part of scipy.io.
#from scipy.io import loadmat 
#
## Beginning at release 7.3 of Matlab, mat files are actually saved using the HDF5 format by default (except if you use the -vX flag at save time, see in Matlab). These files can be read in Python using, for instance, the PyTables or h5py package
#import tables 
#import h5py

In [ ]:
## Get the ggplot - oriented Python library
#!sudo -H -S pip3 install -t "/home/haris/anaconda3/lib/python3.7/site-packages/" --upgrade pandas plotnine \
#< /home/haris/Desktop/running/metabolic_network_pipeline_volestipy/my_project_virtual_env/error.txt
#print("*** The ggplot for Python library has now been installed *** \n\n")

In [ ]:
#mat_file_with_loadmat = loadmat('/home/haris/Downloads/e_coli_core.mat')

While this one is in case of ```.mat``` files created by releases of Matlab later after the 7.3
This command will not run in any other case.
For this demo we will use the ```loadmat``` option.

In [ ]:
# mat_file_with_tables = h5py.File('/home/haris/Downloads/e_coli_core.mat')

Now you can see your metabolic network. 

In [ ]:
print(mat_file_with_loadmat)

In [ ]:
#data_from_mat = mat_file_with_loadmat
#print("the data type of the variable with the network as it was read is: " + str(type(data_from_mat)))

#s_matrix = data_from_mat.keys()
#print("\nthe keys of this dictionaries are: ")
#print(s_matrix)

#e_coli_np_void = data_from_mat['e_coli_core'][0][0]
#print("\nHowever, if we keep the key:value pair of this dictionary, called 'e_coli_core' where the necessary \
#information is located, we can see that its type is: " + str(type(e_coli_np_void)) + "\n\n")

#print("number of dimensions of the np.void data type equals to:" + str(e_coli_np_void.ndim) + "\n")

#print(type(e_coli_np_void))
#print(len(e_coli_np_void))



## metabolites
#print(type(e_coli_np_void[0]))
#print(e_coli_np_void[0].shape)

#print(type(e_coli_np_void[0][0]))
#print(e_coli_np_void[0][0].shape)

#print(e_coli_np_void[0][:3,])
#metabolites = [item[0][0] for item in e_coli_np_void[0]]
#print(metabolites)

## genes
#print(type(e_coli_np_void[4]))
#print(e_coli_np_void[4].shape)
#print(e_coli_np_void[4][:3,])

## reactions
#print(type(e_coli_np_void[7]))
#print(e_coli_np_void[7].shape)
#print(e_coli_np_void[7][:3,])

In [ ]:
#for key,value in data_from_mat.items():
#    print(str(key) + "\t" + str(value))
#    print("\n\n\n\n\n\n")

In [ ]:
rounding_output_max_ellipsoid = hp.rounding(rounding_method = "max_ellipsoid", 
                                            inner_point = max_ball_center_point, 
                                            radius = max_ball_radius)